#GANSPACE - Discovering Interpretable GAN Controls 

Using https://github.com/harskish/ganspace to find latent directions in a stylegan2 model. (This could easily be ported to other models, if anyone implements it please get in touch, and i'll add it to the notebook!)

Notebook put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman)



In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/



## Setup
Hit play on all the cells below, and everything should run smoothly. The install takes around a minute and a half - go grab yourself a cup of tea.



In [0]:
# Clone git
!git clone https://github.com/harskish/ganspace
%cd ganspace


In [0]:
#@title Install remaining packages
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))

!pip install fbpca pyopengltk Ninja
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
%cd models/stylegan2/stylegan2-pytorch/op
!python setup.py install
!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
%cd "/content/ganspace"

# Convert model weights

If you have a tensorflow model you want to use Ganspace on - convert it to a pytorch model below.

(skip this step if you already have a pytorch model)

In [0]:
!gdown --id 1UlDmJVLLnBD9SnLSMXeiZRO6g-OMQCA_ -O /content/ffhq.pkl

In [0]:
%cd "/content"
!git clone https://github.com/skyflynil/stylegan2
%cd ganspace

The convert weight script takes two arguments: 

```
--repo - Path to tensorflow stylegan2 repo
       - Path to your model
```



In [0]:
!python /content/ganspace/models/stylegan2/stylegan2-pytorch/convert_weight.py --repo="/content/stylegan2/" "/content/ffhq.pkl" #convert weights

In [0]:
!cp "/content/ganspace/ffhq.pt" "/content/drive/My Drive/ML/stylegan_models" #copy pytorch model to your drive

# Run PCA Analysis

From here, open models/wrappers.py, and edit the stylegan2 configs dict on line 110 to include your model and its corresponding resolution.

I.E from

        # Image widths
        configs = {
            # Converted NVIDIA official
            'ffhq': 1024,
            'car': 512,
            'cat': 256,
            'church': 256,
            'horse': 256,
            # Tuomas
            'bedrooms': 256,
            'kitchen': 256,
            'places': 256,
        }

to 

        # Image widths
        configs = {
            # Converted NVIDIA official
            'your_model': your_resolution,
            'ffhq': 1024,
            'car': 512,
            'cat': 256,
            'church': 256,
            'horse': 256,
            # Tuomas
            'bedrooms': 256,
            'kitchen': 256,
            'places': 256,
        }

Then copy your pytorch model over to your drive account or any other hosting platform, and add the direct download link to the checkpoints dict in the download_checkpoint function on line 136.

    def download_checkpoint(self, outfile):
        checkpoints = { 'yourmodel': 'https://drive.google.com/yourmodel',
            'horse': 'https://drive.google.com/uc?export=download&id=18SkqWAkgt0fIwDEf2pqeaenNi4OoCo-0',
            'ffhq': 'https://drive.google.com/uc?id=12yYXZymadSIj74Yue1Q7RrlbIqrXggo3',
            'church': 'https://drive.google.com/uc?export=download&id=1HFM694112b_im01JT7wop0faftw9ty5g',
            'car': 'https://drive.google.com/uc?export=download&id=1iRoWclWVbDBAy5iXYZrQnKYSbZUqXI6y',
            'cat': 'https://drive.google.com/uc?export=download&id=15vJP8GDr0FlRYpE8gD7CdeEz2mXrQMgN',
            'places': 'https://drive.google.com/uc?export=download&id=1X8-wIH3aYKjgDZt4KMOtQzN1m4AlCVhm',
            'bedrooms': 'https://drive.google.com/uc?export=download&id=1nZTW7mjazs-qPhkmbsOLLA_6qws-eNQu',
            'kitchen': 'https://drive.google.com/uc?export=download&id=15dCpnZ1YLAnETAPB0FGmXwdBclbwMEkZ'
        }




##Options


```
Command line paramaters:
  --model      one of [ProGAN, BigGAN-512, BigGAN-256, BigGAN-128, StyleGAN, StyleGAN2]
  --class      class name; leave empty to list options
  --layer      layer at which to perform PCA; leave empty to list options
  --use_w      treat W as the main latent space (StyleGAN / StyleGAN2)
  --inputs     load previously exported edits from directory
  --sigma      number of stdevs to use in visualize.py
  -n           number of PCA samples
  -b           override automatic minibatch size detection
  -c           number of components to keep

```




In [0]:
%cd ../ganspace/

In [0]:
model_name = 'StyleGAN2' 
model_class = 'ffhq' #this is the name of your model in the configs
num_components = 80

In [0]:
#Check layers available for analysis
!python visualize.py --model $model --class $model_class --use_w

Add chosen layer in as --layer argument:

In [0]:
!python visualize.py --model $model_name --class $model_class --use_w --layer=style -c $num_components

In [0]:
!python visualize.py --model=$model_name --class=$model_class --use_w --layer="style" -b=500 --batch --video #add -video to generate videos

In [0]:
!python visualize.py --model=StyleGAN2 --class=ffhq --use_w --layer=style -b=10000

In [0]:
!zip -r samples.zip "/content/ganspace/out/StyleGAN2-ffhq" #zip up samples for download

In [0]:
%cp -r "/content/ganspace/cache/components" "/content/drive/My Drive/ML/stylegan2/comps" #copying components over to google drive

# Explore Directions!

After running visualize.py, your components will be stored in an npz file in /content/ganspace/cache/components/ - below the npz file is unpacked, and a component/direction is chosen at random. 

Using the UI, you can explore the latent direction and give it a name, which will be appeneded to the named_directions dictionary and saved as 'direction_name.npy' for later use.


In [0]:
# Load model
from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  layer='style',
  output_class='ffhq',
  components=80,
  use_w=True,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

named_directions = {} #init named_directions dict to save directions

In [6]:
#@title Load a component at random

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
            
#load one at random 
num = np.random.randint(20)
if num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[num]
random_dir_stdev = latent_stdevs[num]

print(f'Loaded Component No. {num}')


Loaded Component No. 0


In [7]:
#@title Run UI
from ipywidgets import fixed

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  filename += ".npy"
  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def display_sample_pytorch(seed, truncation, direction, distance, scale, start, end, disp=True, save=None, noise_spec=None):
    # blockPrint()
    with io.capture_output() as captured:
      w = model.sample_latent(1, seed=seed).cpu().numpy()

      model.truncation = truncation
      w = [w]*model.get_max_latents() # one per layer
      for l in range(start, end):
          w[l] = w[l] + direction * distance * scale

      #save image and display
      out = model.sample_np(w)
      final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)

    if disp:
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

def generate_mov(seed, truncation, direction_vec, scale, layers, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))


seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=0, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=18, description='end layer: ', continuous_update=False)

# Make sure layer range is valid
def update_range_start(*args):
  end_layer.min = start_layer.value
def update_range_end(*args):
  start_layer.max = end_layer.value
start_layer.observe(update_range_start, 'value')
end_layer.observe(update_range_end, 'value')

text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, scale, start_layer, end_layer, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer})

display(ui, out)
text.on_submit(name_direction)

Output()

In [0]:
#script to generate a movie moving back and forth along the direction

direction_name = 'raise_eyebrows'
num_samples = 5

loc = named_directions[direction_name][0]
for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = 2, layers=range(named_directions[direction_name][1], named_directions[direction_name][2]), n_frames = 20, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

In [0]:
#@title Select from named directions

from IPython.display import display, clear_output

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    random_dir = latent_dirs[named_directions[select_variable.value][0]]
    start_layer = named_directions[select_variable.value][1]
    end_layer = named_directions[select_variable.value][2]
    print(start_layer, end_layer)
    out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})
    display(select_variable)
    display(ui, out)

random_dir = latent_dirs[named_directions[select_variable.value][0]]
start_layer = named_directions[select_variable.value][1]
end_layer = named_directions[select_variable.value][2]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance, scale])
ui = widgets.VBox([bot_box])
out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})

display(select_variable)
display(ui, out)

select_variable.observe(set_direction, names='value')

